In [1]:
import cv2
from cnn import CNN
import tensorflow as tf
cnn = CNN()

In [2]:
if __name__ == '__main__':
    using_cv = True
    #using_cv = False
    
    data_list = [
        {"folder":"img_face","label":"0"},
        {"folder":"img_face_non_obama","label":"1"}
    ]
    
    # test data    
    test_list = [
        {"folder":"img_face_test", "label":"0"},
        {"folder":"img_face_test_non_obama", "label":"1"}
    ]
    if not using_cv:
        _train_image, train_label, _ = cnn.read_image(data_list)
      
        sess = tf.Session()
        sess.run(tf.initialize_all_variables())  
        train_image = []
        for img in _train_image:
            print img
            i = sess.run(img)
            train_image.append(i)
            
        _test_image , test_label , test_image_list= cnn.read_image(test_list)  
        test_image = []
        for img in _test_image:
            i = sess.run(img)
            test_image.append(i)
    else:
        _train_image, train_label, _ = cnn.read_image_cv(data_list)
        train_image = _train_image
        _test_image , test_label , test_image_list = cnn.read_image_cv(test_list)  
        test_image = _test_image
    
    with tf.Graph().as_default():
        
        images_placeholder = tf.placeholder("float", shape=(None, cnn.IMAGE_PIXELS))
        labels_placeholder = tf.placeholder("float", shape=(None, cnn.NUM_CLASSES))
        # dropout
        keep_prob = tf.placeholder("float")

    
        softmax = cnn.inference(images_placeholder, keep_prob)
        loss_value = cnn.loss(softmax, labels_placeholder)
        train_op = cnn.training(loss_value, cnn.LEARNING_RATE)
        acc = cnn.accuracy(softmax, labels_placeholder)

        
        saver = tf.train.Saver()

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        
        # for TensorBoard
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter (cnn.LOG_DIR, sess.graph)
        
        # 訓練の実行
        for step in range(cnn.MAX_STEPS):
            for i in range(len(train_image)/cnn.BATCH_SIZE):
                # batch_size分の画像に対して訓練の実行
                batch = cnn.BATCH_SIZE*i
                sess.run(train_op, feed_dict={
                  images_placeholder: train_image[batch:batch+cnn.BATCH_SIZE],
                  labels_placeholder: train_label[batch:batch+cnn.BATCH_SIZE],
                  keep_prob: 0.5})

            # 1 step終わるたびに精度を計算する
            #train_accuracy = sess.run(acc, feed_dict={
            #    images_placeholder: train_image,
            #    labels_placeholder: train_label,
            #    keep_prob: 1.0})
            #print "step %d, training accuracy %g"%(step, train_accuracy)
               
            train_loss = sess.run(loss_value,  feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0})
            print "step %d, xentropy %s"%(step, str(train_loss))
            
            if step % 10 == 0 and step > 0 and train_loss < 5.0:
                cnn.display_test_prob(sess, softmax, test_image, "オバマ", images_placeholder,keep_prob)

            # 1 step終わるたびにTensorBoardに表示する値を追加する
            summary_str = sess.run(summary_op, feed_dict={
                images_placeholder: train_image,
                labels_placeholder: train_label,
                keep_prob: 1.0})
            summary_writer.add_summary(summary_str, step)

    # 訓練が終了したらテストデータに対する精度を表示
    print "test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
       keep_prob: 1.0})
    

    # 最終的なモデルを保存
    save_path = saver.save(sess, "model.ckpt")

step 0, xentropy 2624.95
step 1, xentropy 2624.95
step 2, xentropy 2624.95
step 3, xentropy 2624.95
step 4, xentropy 2624.95
step 5, xentropy 2624.95
step 6, xentropy 2624.95
step 7, xentropy 2624.95
step 8, xentropy 2624.95


KeyboardInterrupt: 